## Обухов Илья ПИ225
# Семинар 8 21.05.23

## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [8]:
import random
N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [a + 100 for a in A]
av_B = sum(B) / N
print(av_B)

599.897738


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [21]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.rand(2000000, 4), columns=['1', '2', '3', '4'])
df['key'] = [chr(np.random.randint(97, 122)) for i in range(2000000)]
res = df[df['key'].str.contains('^[a-eA-E]')]
print(res)

                1         2         3         4 key
8        0.390895  0.455269  0.005959  0.802900   a
9        0.343274  0.372798  0.180353  0.986909   e
10       0.729821  0.961028  0.079245  0.607695   d
12       0.964955  0.306013  0.576941  0.644787   c
14       0.870469  0.576169  0.972405  0.064861   e
...           ...       ...       ...       ...  ..
1999968  0.323718  0.894155  0.185776  0.780685   a
1999981  0.206165  0.263084  0.663382  0.786747   b
1999989  0.413212  0.649599  0.258448  0.615082   b
1999995  0.808879  0.128536  0.144353  0.460786   b
1999996  0.980360  0.008147  0.318783  0.069475   a

[401673 rows x 5 columns]


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [49]:
import pandas as pd
import time
recipes = pd.read_csv('recipes_sample.csv', index_col=0, delimiter=',')
reviews = pd.read_csv('reviews_sample (1).csv', index_col=0, delimiter=',')
reviews['date'] = pd.to_datetime(reviews['date'])
def m1(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            sumall += row['rating']
            counter += 1
    return sumall / counter
def m2(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.loc[reviews['date'].dt.year == 2010].iterrows():
        sumall += row['rating']
        counter += 1
    return sumall / counter
def m3(reviews):
    filtered_reviews = reviews.loc[reviews['date'].astype(str).str.contains('2010')]
    return filtered_reviews['rating'].mean()
print(f'1) {m1(reviews)}')
print(f'2) {m2(reviews)}')
print(f'3) {m3(reviews)}')
print('\n')
start_time = time.time()
res1 = m1(reviews)
end_time = time.time()
print(f'm1: {res1}, time - {end_time - start_time}')
start_time = time.time()
res2 = m2(reviews)
end_time = time.time()
print(f'm2: {res2}, time - {end_time - start_time}')
start_time = time.time()
res3 = m3(reviews)
end_time = time.time()
print(f'm3: {res3}, time - {end_time - start_time}')

1) 4.4544402182900615
2) 4.4544402182900615
3) 4.4544402182900615


mean1: 4.4544402182900615, time - 4.450187683105469
mean2: 4.4544402182900615, time - 0.4156954288482666
mean3: 4.4544402182900615, time - 0.8187837600708008


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [63]:
print('Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимся\
только по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемых\
данных, вследствие чего сильно увеличивает время выполнения функции')
print('\n')
%lprun -f mean1 mean1(reviews)
def mean1_2(reviews):
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    count = len(filtered_reviews)
    if count > 0:
        return filtered_reviews['rating'].apply(lambda x: float(x)).sum() / count
    else:
        return None
%lprun -f mean1_2 mean1_2(reviews)
print('Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так как\
в памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись')

Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимсятолько по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемыхданных, вследствие чего сильно увеличивает время выполнения функции


Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так какв памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [83]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)
from collections import defaultdict
def get_word_reviews_count2(df):
    word_reviews = defaultdict(list)
    word_reviews_count = defaultdict(int)
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        for word in review.split():
            word_reviews[word].append(recipe_id)
    for word in word_reviews:
        word_reviews_count[word] = len(word_reviews[word])
%lprun -f get_word_reviews_count2 get_word_reviews_count2(reviews)